# DAY 3: Student version

**Machine Learning NLP**

The goal of this session is to improve the search engine using NLP features.

This notebook guides you through different techniques to explore. It is expected of you to be inventive and improve the techniques introduced. 

First, let's import the useful packages and load the data.

## Installs

In [20]:
! pip install sentence-transformers --quiet

## Imports

In [21]:
import os
import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import find

from sentence_transformers import SentenceTransformer

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load Data

In [22]:
# Only if you use Colab
from google.colab import drive
drive.mount('/content/drive')

import os

# TODO:
DATA_PATH = '/content/drive/MyDrive/EI/data' 

# CORR:
# DATA_PATH = '/content/drive/MyDrive/TP Centrale/data'
     

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")
posts

,Id,PostTypeId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,...,ClosedDate,ContentLicense,AcceptedAnswerId,LastEditorUserId,LastEditDate,ParentId,OwnerDisplayName,CommunityOwnedDate,LastEditorDisplayName,FavoriteCount
0,5,1,2014-05-13T23:58:30.457,9,898.0,<p>I've always been interested in machine lear...,5.0,2014-05-14T00:36:31.077,How can I do simple machine learning without h...,<machine-learning>,...,2014-05-14T14:40:25.950,CC BY-SA 3.0,NaN,NaN,None,NaN,None,None,None,NaN
1,7,1,2014-05-14T00:11:06.457,4,478.0,"<p>As a researcher and instructor, I'm looking...",36.0,2014-05-16T13:45:00.237,What open-source books (or other materials) pr...,<education><open-source>,...,2014-05-14T08:40:54.950,CC BY-SA 3.0,10.0,97.0,2014-05-16T13:45:00.237,NaN,None,None,None,NaN
2,9,2,2014-05-14T00:36:31.077,5,NaN,"<p>Not sure if this fits the scope of this SE,...",51.0,2014-05-14T00:36:31.077,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,5.0,None,None,None,NaN
3,10,2,2014-05-14T00:53:43.273,13,NaN,"<p>One book that's freely available is ""The El...",22.0,2014-05-14T00:53:43.273,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,7.0,None,None,None,NaN
4,14,1,2014-05-14T01:25:59.677,26,1901.0,<p>I am sure data science as will be discussed...,66.0,2020-08-16T13:01:33.543,Is Data Science the Same as Data Mining?,<data-mining><definitions>,...,None,CC BY-SA 3.0,29.0,322.0,2014-06-17T16:17:20.473,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75722,119962,1,2023-03-04T20:06:06.820,0,8.0,<p>I am implementing a neural network of arbit...,147597.0,2023-03-04T20:22:12.523,Back Propagation on arbitrary depth network wi...,<neural-network><backpropagation>,...,None,CC BY-SA 4.0,NaN,147597.0,2023-03-04T20:22:12.523,NaN,None,None,None,NaN
75723,119963,1,2023-03-04T20:12:19.677,0,10.0,<p>I am using KNN for a regression task</p>\n<...,147598.0,2023-03-04T20:12:19.677,Evaluation parameter in knn,<regression><k-nn>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75724,119964,1,2023-03-05T00:14:12.597,0,7.0,<p>I have developed a small encoding algorithm...,44581.0,2023-03-05T00:14:12.597,Can I use zero-padded input and output layers ...,<deep-learning><convolutional-neural-network>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75725,119965,1,2023-03-05T00:43:12.213,0,5.0,"<p>To my understanding, optimizing a model wit...",84437.0,2023-03-05T00:43:12.213,Why does cross validation and hyperparameter t...,<cross-validation><hyperparameter-tuning>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN


## Data Cleaning

Implement a function to clean the posts. 

You can reuse what you have used in the Day 2 notebook or improve it.

In [24]:
def clean_post(text:str)->str:
  res=""
  in_tag=False
  next=False
  for lettre in text:
    if lettre=='<' and not in_tag:
      in_tag=True
    elif lettre=='>' and in_tag:
      in_tag=False
    elif lettre=='\n' or lettre=='\t':
      next=True
    elif not in_tag:
      if next:
        res+=" " + str(lettre)
        next=False
      else:
        res+=str(lettre)
  return res

In [25]:
clean_posts = posts[['Id','Body']]
clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(clean_post)
clean_posts

<ipython-input-25-f2d19e1d3215>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(clean_post)


,Id,Body,Clean Body
0,5,<p>I've always been interested in machine lear...,I've always been interested in machine learnin...
1,7,"<p>As a researcher and instructor, I'm looking...","As a researcher and instructor, I'm looking fo..."
2,9,"<p>Not sure if this fits the scope of this SE,...","Not sure if this fits the scope of this SE, bu..."
3,10,"<p>One book that's freely available is ""The El...","One book that's freely available is ""The Eleme..."
4,14,<p>I am sure data science as will be discussed...,I am sure data science as will be discussed in...
...,...,...,...
75722,119962,<p>I am implementing a neural network of arbit...,I am implementing a neural network of arbitrar...
75723,119963,<p>I am using KNN for a regression task</p>\n<...,I am using KNN for a regression task It's like...
75724,119964,<p>I have developed a small encoding algorithm...,I have developed a small encoding algorithm th...
75725,119965,"<p>To my understanding, optimizing a model wit...","To my understanding, optimizing a model with k..."


You can also implement a function that cleans the user's query (the query). 

This step is optionnal (if you don't think that it is necessary, just return the query)

In [26]:
def clean_query(text:str)->str:
    cleaned_query = clean_post(text)
    return cleaned_query

## Text specific metadata

What metadata can you get from the text at your disposal ? Which ones are relevant ? 

In [27]:
#TODO
posts.keys()

Index(['Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body',
       'OwnerUserId', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'ClosedDate', 'ContentLicense', 'AcceptedAnswerId',
       'LastEditorUserId', 'LastEditDate', 'ParentId', 'OwnerDisplayName',
       'CommunityOwnedDate', 'LastEditorDisplayName', 'FavoriteCount'],
      dtype='object')

## Classic Preprocessing

The goal for this part is to implement a classic vectorization (Count vectorizer, tfidf...).

You can do it on your own or use scikit-learn.

Hints : pay attention to stopwords, additionnal preprocessing steps and techniques of vectoriation


In [28]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(clean_posts["Clean Body"].values)
vectors = vectorizer.transform(clean_posts["Clean Body"].values)

Write a function that applies the same process to the query


In [29]:
def vectorize_query(query : str, vectorizer=vectorizer):
    """vectorizes the query
    Args:
        query (str): query string
        vectorizer (optional): Defaults to vectorizer.

    Returns:
        query vectorized
    """
    query_clean=clean_query(query)
    query_vectorized=vectorizer.transform([query_clean])
    return query_vectorized

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(vectors, vectorize_query("Test de l'EI"))

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.        ],
       [0.12549116],
       [0.        ]])

Determine a way to use this vectorization to suggest the closest items to the entry in the database

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

def vectorizer_search(query : str,
                      vectors=vectors,
                      vectorizer=vectorizer,
                      top=10) -> list:
    query_vector=vectorize_query(query)
    values=cosine_similarity(vectors, query_vector)
    values=[[values[k][0],k] for k in range(len(values))]
    values.sort(reverse=True)
    closest_items = [clean_posts["Clean Body"][x[1]] for x in values[:top]]
    return closest_items

In [32]:
entry = 'what is stochastic gradient descent ?'

In [33]:
 vectorizer_search(entry, top=1)[0]

'As I know, Gradient Descent has three variants which are: 1- Batch Gradient Descent: processes all the training examples for each iteration of gradient descent. 2- Stochastic Gradient Descent: processes one training example per iteration. Hence, the parameters are being updated even after one iteration in which only a single example has been processed. 3- Mini Batch gradient descent: which works faster than both batch gradient descent and stochastic gradient descent. Here, b examples where b &lt; m are processed per iteration. But in some cases they use Stochastic Gradient Descent and they define a batch size for training which is what I am confused about. Also, what about Adam, AdaDelta &amp; AdaGrad, are they all mini-batch gradient descent or not?'

How you can improve this approach ? 

*Answer* here

On pourrait mettre en place un calcul de rareté des termespour mettre en avant les similarités sur les mots rares. Cela pourrait se faire par l'ajout de poids.

## Semantic similarity

There are NLP methods that go further than word-by-word study, by taking into account the context of the terms. There are several methods: Word2vec, Bert.

From the Sentence Transformers documentation: https://www.sbert.net/docs/pretrained_models.html choose the pre-trained model that you think is the most appropriate. Justify your choice.

In [34]:
sentence_transformer_model = 'multi-qa-MiniLM-L6-cos-v1'

Answer Here

⚠ To use Sentence Transformers it is recommended to activate the GPU of google colab.

In [35]:
MODEL_ST = SentenceTransformer(sentence_transformer_model)

Use this algorithm to encode the data in the database

In [36]:
embeddings = MODEL_ST.encode(clean_posts["Clean Body"].values, normalize_embeddings=True)

*If this process is slow, you can save this array in case you need to load it again*

In [37]:
import pickle

with open(os.path.join(DATA_PATH, 'embeddings.pkl'), 'wb') as file:
    pickle.dump(embeddings, file)

In [38]:
# Pour ouvrir le fichier .pkl dans DATA_PATH
with open(os.path.join(DATA_PATH, 'embeddings.pkl'), 'rb') as file:
  embeddings = pickle.load(file)

Make a function that transforms the input

In [39]:
def encode_query(query : str) ->  np.ndarray:
    encoded_query=MODEL_ST.encode(query)
    return encoded_query

Which distance is most relevant to measure the distance between the input and the data?

Answer here

La distance 1-sim(a,b)

Write a function that returns a matrix containing information about the similarity between the query and the data

In [77]:
def similarity(query, sim="Cosine", embeddings=embeddings):
    #TODO
    query_encoded = encode_query(query)
    if sim=="Cosine":
      similarity_matrix = cosine_similarity(embeddings, [query_encoded])
    elif sim=="Pearson":
      avg_q=sum(query_encoded)/len(query_encoded)
      query_encoded=[x-avg_q for x in query_encoded]
      embeddings_avg=[]
      for l in embeddings:
        avg=sum(l)/len(l)
        l_cent=[x-avg for x in l]
        embeddings_avg.append(l_cent)
      embeddings_avg=np.asarray(embeddings_avg)
      similarity_matrix = cosine_similarity(embeddings, [query_encoded])
    return similarity_matrix.transpose()[0]

In [76]:
query_encoded = encode_query(query)
avg_q=sum(query_encoded)/len(query_encoded)
query_encoded=[x-avg_q for x in query_encoded]
query_encoded

[0.07741857958925873,
 0.006459381300639204,
 -0.05548104750024171,
 0.02760871579838901,
 -0.051938058007825326,
 0.03499161718381553,
 0.02142656115783363,
 -0.035292250234950494,
 -0.06052632736073823,
 -0.038394575137961816,
 -0.018832099754680105,
 -0.06789186703072876,
 0.061652607153784324,
 0.004222695119362167,
 -0.01340544609384627,
 -0.05294752972232194,
 0.040268066208492805,
 -0.022273316178787656,
 0.09146999461544662,
 -0.060539950747359704,
 0.0654906172860971,
 0.01176275020433574,
 0.009538180064093163,
 -0.04147497298227639,
 -0.006329981275219628,
 -0.05143829168783517,
 0.022729960139524035,
 0.009216468650530866,
 0.0491639588583818,
 -0.01915402444111199,
 -0.017211550046075292,
 0.012636609244774869,
 -0.023757486734856077,
 0.06300238413704544,
 0.048011104445587684,
 -0.03791676567899079,
 0.012010566759537747,
 0.039935181718002845,
 -0.003011741806138464,
 0.05853757528556495,
 -0.02162999020921082,
 -0.10559571640597672,
 0.03633467746866851,
 -0.0026519188

In [79]:
query = "This query is there to test this search engine"
matrix_similarity = similarity(query, "Pearson", embeddings)

In [80]:
matrix_similarity

array([-0.07798918,  0.03562171,  0.00442115, ...,  0.03057431,
       -0.03794395,  0.04464634])

How do you determine which documents in the data set most closely match the input?

In [72]:
def ordre_en_fonction_similarité(matrix_similarity):
    #TODO
    n=len(matrix_similarity)
    L=[[matrix_similarity[k],k] for k in range(n)]
    L.sort()
    L.reverse()
    ordre = [l[1] for l in L]
    return ordre

In [73]:
assert ordre_en_fonction_similarité([0.6, 0.8, 0.7]) == [1, 2, 0]

Put it all together in a function.

In [83]:
def closest_semantic_doc(query, sim="Cosin", embeddings=embeddings, top_n=10):
    sim = similarity(query, sim, embeddings)
    ordre = ordre_en_fonction_similarité(sim)[:top_n]
    closest_posts = [clean_posts["Clean Body"][k] for k in ordre]
    return closest_posts

In [85]:
closest_semantic_doc(query, "Pearson")

['This is a hyperparameter. And so this is usually approached with some sort of hyperparameters search with cross-validation, like GridSearchCV or RandomizedSearchCV. https://scikit-learn.org/stable/modules/grid_search.html',
 "I want to understand the intent of the customer using his search queries, let's say if a customer is interested in yoga pants, he can either search for yoga pants or exercise pants or workout tights etc. Is there a model that I can use to find out all the search keywords that can be related to yoga pants? ",
 'You could use cross validation with grid search as shown here',
 "I have a very large dataset consisting of approximately 15,000 rows of survey data. The surveys are quantitative summaries of interviews with investors regarding their rankings of financial asset managers. They rate a manager on a scale from 1-5 and we have approximately 5 years of data for each investor on asset manager. The data is indexed as: Investor name | Manager name | Year | Rating p

What methods could be used to improve the recommendations of this algorithm?

Answer here

On peut changer le model ou la distance utilisée

## Text clustering (BONUS)

We can use topic modeling techniques to identify groups of texts among our document base and classify the input to restrict the application of the proximity calculations seen previously.

#### LDA

Latent Dirichlet Allocation is a topic modeling algorithm that allows soft clustering. Soft clustering means that the LDA does not allocate an input to a cluster, but gives a probabilistic score for each identified cluster. This decomposition allows to identify topics within the documents. 

In order to compute this algorithm, you need to vectorize your data (you can use the one you have already done previously or make another one).

In [91]:
# Vectorize document using TF-IDF
vectorizer_lda = TfidfVectorizer(stop_words='english')

# Fit and Transform the documents
train_data = vectorizer_lda.fit_transform(clean_posts["Clean Body"].values)

You can use Gensim or scikit-learn to compute LDA.

In [109]:
#TODO
lda_model = LatentDirichletAllocation()
lda_model.fit(train_data)

LatentDirichletAllocation()

Assign a main topic to each document

In [110]:
topic_documents = lda_model.transform(train_data)

In [111]:
topic_documents

array([[0.01560078, 0.01560078, 0.01560333, ..., 0.74946347, 0.01560078,
        0.01560078],
       [0.01724153, 0.01724153, 0.0172432 , ..., 0.8448239 , 0.01724153,
        0.01724153],
       [0.0378273 , 0.01304815, 0.01304907, ..., 0.81405751, 0.01304815,
        0.01304815],
       ...,
       [0.01030455, 0.01030455, 0.01030506, ..., 0.90725489, 0.01030455,
        0.01030455],
       [0.01327687, 0.01327687, 0.01327737, ..., 0.84968945, 0.01327687,
        0.01327687],
       [0.01189033, 0.01189027, 0.0118921 , ..., 0.21305661, 0.01189051,
        0.01189028]])

In [144]:
def get_topic_query(query, vectorizer=vectorizer_lda, lda_model=lda_model) -> int:
    vectorized_query = vectorize_query(query, vectorizer)
    Ltopics = lda_model.transform(vectorized_query)[0]
    Ltopics = list(Ltopics)
    topics =  Ltopics.index(max(Ltopics))
    return topics

In [151]:
get_topic_query("Licorne dinosaure et au milieu")

7

Write a function that assigns a topic to the query


## Merge methods

Write an algorithm to merge the methods seen. Which methods to use? How can you check if they are relevant ?

Answer here

In [ ]:
def nlp_search_algorithm(query,
                         topic_documents=topic_documents,
                         vectors=vectors,
                         vectorizer=vectorizer,
                         vectorizer_lda=vectorizer_lda,
                         lda_model=lda_model,
                         embeddings=embeddings,
                         top_n=10
                         )->list:
    #TODO
    topic=get_topic_query("Licorne dinosaure et au milieu",vectorizer, lda_model)


    return matching_posts




Once you have a list of possible results, you can it: (you can use one of the ranking algorithms you have previously done or make up a new one)

In [ ]:
def rank(possible_results):
    #to_do
    return possible_results

## Incorporation in the search engine

### Addition of metadata

You must now have a new set of metadata and data to add to your original index. You can load the index you had as a result of Day 2 and today's work to it. 

In [ ]:
#load previous data 

#TODO 

In [ ]:
# add the new data to the previous index

Hint : if you have changed the preprocessing function at the beginning of this notebook make sure to update the Clean Body attribute

### Adaptation to the index format

Adapt your nlp search results to the index format

In [ ]:
def nlp_search_in_index(query,
                        index,
                        args):

    return matching_posts
  

### Compare the new searching and ranking method to the previous ones

Compare in terms of efficiency (precision, completeness, speed, memory usage)

### merge all methods to make an efficient search algorithm